In [21]:
import pandas as pd
import json
import numpy as np

In [22]:
#De Json a dataframe
data = pd.read_json("datos_crudos/datos_bocas_del_toro_crudos_2024.json")

#Eliminar columnas
df = pd.DataFrame(data)
df.drop(['snow', 'pres', 'tsun', 'wpgt'], axis='columns', inplace=True)


In [23]:
print(df.dtypes)

time     object
tavg    float64
tmin    float64
tmax    float64
prcp    float64
wdir    float64
wspd    float64
dtype: object


In [24]:
#Limpieza de datos
df_bocas_del_toro = df.drop(columns=["Día","Año", "Mes"], errors='ignore')
df_bocas_del_toro.replace(["", "-", "NA", "N/A", "n/a"], np.nan, inplace=True)
# Convertir la columna Date a datetime y dejar solo año-mes-día
df_bocas_del_toro["time"] = pd.to_datetime(df_bocas_del_toro["time"], errors="coerce").dt.strftime("%Y-%m-%d")

In [25]:
# Porcentaje de valores faltantes
faltantes = df_bocas_del_toro.isna().sum()
print("Valores faltantes por columna:")
print(faltantes[faltantes > 0])
porcentaje_faltantes = (df_bocas_del_toro.isna().sum() / len(df_bocas_del_toro)) * 100
print("\nPorcentaje de valores faltantes:")
print(porcentaje_faltantes[porcentaje_faltantes > 0].round(2))

Valores faltantes por columna:
tavg     12
tmin     12
tmax     12
prcp     12
wdir    366
wspd     12
dtype: int64

Porcentaje de valores faltantes:
tavg      3.28
tmin      3.28
tmax      3.28
prcp      3.28
wdir    100.00
wspd      3.28
dtype: float64


In [26]:
#Limpieza de las columnas
for col in df_bocas_del_toro:
    if col != 'time':  # Evitar convertir la columna de fechas
        df_bocas_del_toro[col] = pd.to_numeric(df_bocas_del_toro[col], errors='coerce')

# Imputar temperaturas con la media
for col in ["tmax", "tmin", "tavg"]:
    df_bocas_del_toro[col] = df_bocas_del_toro[col].fillna(df_bocas_del_toro[col].mean())

# Imputar precipitaciones con 0 = no llovió)
for col in ["prcp"]:
    df_bocas_del_toro[col] = df_bocas_del_toro[col].fillna(0)

# Imputar viento con la media 
for col in ["wspd"]:
    df_bocas_del_toro[col] = df_bocas_del_toro[col].fillna(df_bocas_del_toro[col].mean())

# Imputar texto faltante con la moda (valor más frecuente)
for col in ["wdir"]:
    if df_bocas_del_toro[col].isna().sum() > 0:
        moda = df_bocas_del_toro[col].mode()
        if not moda.empty:
            df_bocas_del_toro[col] = df_bocas_del_toro[col].fillna(moda[0])

columnas_numericas = df_bocas_del_toro.select_dtypes(include=["number"]).columns
df_bocas_del_toro[columnas_numericas] = df_bocas_del_toro[columnas_numericas].round(1)

In [27]:
#Cambiar el nombre de las columnas y agregar dos adicionales
columnas_estandar= {
    'time': 'Date',
    'tmax': 'Tmax',
    'tmin': 'Tmin',
    'tavg': 'Tavg',
    'prcp': 'Prcp',
    'wspd': 'Wspd',
    'wdir': 'Wdir'
}

# Renombrar columnas y eliminar las demás
df_bocas_del_toro= df_bocas_del_toro.rename(columns=columnas_estandar)  
df_bocas_del_toro = df_bocas_del_toro[list(columnas_estandar.values())] 
df_bocas_del_toro["Estacion"] = "La Cabaña"
df_bocas_del_toro["Provincia"] = "Bocas del Toro"

In [28]:
print(df_bocas_del_toro)



           Date  Tmax  Tmin  Tavg  Prcp  Wspd  Wdir   Estacion       Provincia
0    2024-01-01  30.0  24.0  25.7   8.2  15.5   NaN  La Cabaña  Bocas del Toro
1    2024-01-02  30.0  23.0  26.1   1.2  16.0   NaN  La Cabaña  Bocas del Toro
2    2024-01-03  29.0  23.1  25.9   1.1  10.2   NaN  La Cabaña  Bocas del Toro
3    2024-01-04  29.2  23.8  25.8   2.3  12.6   NaN  La Cabaña  Bocas del Toro
4    2024-01-05  29.5  22.9  25.7   1.7  10.0   NaN  La Cabaña  Bocas del Toro
..          ...   ...   ...   ...   ...   ...   ...        ...             ...
361  2024-12-27  28.0  23.2  25.5   1.9  10.6   NaN  La Cabaña  Bocas del Toro
362  2024-12-28  28.6  23.6  25.9   1.4   9.4   NaN  La Cabaña  Bocas del Toro
363  2024-12-29  26.0  23.5  24.8  22.1   8.6   NaN  La Cabaña  Bocas del Toro
364  2024-12-30  28.0  23.0  25.0  19.2  10.0   NaN  La Cabaña  Bocas del Toro
365  2024-12-31  27.9  23.0  24.9   6.9  11.9   NaN  La Cabaña  Bocas del Toro

[366 rows x 9 columns]


In [30]:
#Pasar a CSV
df_bocas_del_toro.to_csv("datos_limpios/datos_limpios_bocas_del_toro_202444.csv", index=False)